In [ ]:
import os, random, matplotlib, pickle, yaml, hdbscan
import numpy as np
from sklearn.utils import shuffle
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib as mpl
from matplotlib.ticker import FormatStrFormatter
from matplotlib.colors import ListedColormap
from glob2 import glob

In [ ]:
skeleton = [
    [0,1,2,3],
    [4,5,6,7],
    [8,9,10],
    [11,12,13],
    [14,15,16],
    [17,18,19,20],
    [21,22,23],
    [24,25,26],
    [27,28,29]
]
skeleton_color=['tab:blue','tab:orange','tab:green','tab:red','tab:purple','tab:brown','tab:pink','tab:gray','tab:olive', 'tab:cyan']

In [ ]:
result_path = "../results/round3"
with open(f"{result_path}/INFO.yaml") as f:
    INFO = yaml.load(f, Loader=yaml.FullLoader)
    INFO_values = list(INFO.values())
    INFO_values.sort(key=lambda x: x['order'])

In [ ]:
config_path = ".."
with open(f"{config_path}/config.yaml") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [ ]:
root_path = ".."
rot_bp, angles, limbs = [], [], []
angle_power, limb_power = [], []
all_embed = []
all_postural_embed, marker_postural_embed, angle_postural_embed, limb_postural_embed = [], [], [], []
all_kinematic_embed, marker_kinematic_embed, limb_kinematic_embed, angle_kinematic_embed = [], [], [], []

for file in tqdm(INFO_values):
    rot_bp.append( np.load(f"{root_path}/{file['directory']}/rotated_bodypoints.npy") )
    angles.append( np.load(f"{root_path}/{file['directory']}/angles.npy") )
    limbs.append( np.load(f"{root_path}/{file['directory']}/limbs.npy") )
    angle_power.append( np.load(f"{root_path}/{file['directory']}/angle_power.npy") )
    limb_power.append( np.load(f"{root_path}/{file['directory']}/limb_power.npy") )
    
    all_embed.append( np.load(f"{root_path}/{file['directory']}/all_embeddings.npy") )
    
    all_postural_embed.append( np.load(f"{root_path}/{file['directory']}/all_postural_embeddings.npy") )
#     marker_postural_embed.append( np.load(f"{root_path}/{file['directory']}/marker_postural_embeddings.npy") )
#     angle_postural_embed.append( np.load(f"{root_path}/{file['directory']}/angle_postural_embeddings.npy") )
#     limb_postural_embed.append( np.load(f"{root_path}/{file['directory']}/limb_postural_embeddings.npy") )
    
    all_kinematic_embed.append( np.load(f"{root_path}/{file['directory']}/all_kinematic_embeddings.npy") )
#     marker_kinematic_embed.append( np.load(f"{root_path}/{file['directory']}/marker_kinematic_embeddings.npy") )
#     limb_kinematic_embed.append( np.load(f"{root_path}/{file['directory']}/marker_limb_embeddings.npy") )
#     angle_kinematic_embed.append( np.load(f"{root_path}/{file['directory']}/marker_angle_embeddings.npy") )
    
rot_bp = np.concatenate(rot_bp)
angles = np.concatenate(angles)
limbs = np.concatenate(limbs)
angle_power = np.concatenate(angle_power)
limb_power = np.concatenate(limb_power)

all_embed = np.concatenate(all_embed)

all_postural_embed = np.concatenate(all_postural_embed)
# marker_postural_embed = np.concatenate(marker_postural_embed)
# angle_postural_embed = np.concatenate(angle_postural_embed)
# alllimb_postural_embed_embed = np.concatenate(limb_postural_embed)

all_kinematic_embed = np.concatenate(all_kinematic_embed)
# marker_kinematic_embed = np.concatenate(marker_kinematic_embed)
# limb_kinematic_embed = np.concatenate(limb_kinematic_embed)
# angle_kinematic_embed = np.concatenate(angle_kinematic_embed)

<hr>
<hr>

# Morlet Angle

In [ ]:
num_clusters = int(np.max(labels) + 1)

fig, ax = plt.subplots(5,5,figsize=(20,30), dpi=400)
for i_cluster in range(num_clusters):
#     idx_cluster = (labels == i_cluster) & (clusterer.probabilities_>0.7)
    idx_cluster = (labels == i_cluster)
    num_fr_in_cluster = len(np.where(idx_cluster)[0])
    
    xtick_idx = np.arange(1, len(freq), 3)
    
    # Create figure
    ax[i_cluster//5, i_cluster%5].imshow(np.mean(angle_power[idx_cluster,:,:], axis=0).T, norm = matplotlib.colors.LogNorm(vmin=np.min(angle_power), vmax=np.max(angle_power)), 
              vmin=0.01, vmax=np.max(angle_power[:,:,:]), cmap='hot')
    
    # Figure Setting Specific
    if i_cluster%5 == 0: 
        ax[i_cluster//5, i_cluster%5].set(xticks=xtick_idx, xticklabels=np.around(freq[xtick_idx], 1), 
               yticks=np.arange(len(config['angle_labels'])), yticklabels=config['angle_labels'],
               aspect=1.1, xlabel="Frequency", 
               title=f"Avg Power Cluster {i_cluster} ({num_fr_in_cluster} frames)")
    else:
        ax[i_cluster//5, i_cluster%5].set(xticks=xtick_idx, xticklabels=np.around(freq[xtick_idx], 1), yticks=[],
               aspect=1.1, xlabel="Frequency", 
               title=f"Avg Power Cluster {i_cluster} ({num_fr_in_cluster} frames)")
plt.show()

In [ ]:
marker_idx = 4

num_clusters = int(np.max(labels) + 1)
marker_psd = []

for i_cluster in range(num_clusters):
    idx_cluster = (labels == i_cluster)
    marker_psd.append( np.mean(angle_power[idx_cluster,:,marker_idx], axis=0) )
marker_psd = np.stack(marker_psd)

fig, ax = plt.subplots(figsize=(5,6))
ax.imshow(marker_psd.T, norm = matplotlib.colors.LogNorm(), 
          vmin=0.01, vmax=np.max(angle_power[:,:,marker_idx]), cmap='hot')
ax.set(
    yticks=np.arange(1, len(freq), 3), yticklabels=np.around(freq[xtick_idx], 1), 
    aspect=1.1, ylabel="Frequency", xlabel="Cluster" ,
    title=f"Avg Power Cluster {config['angle_labels'][marker_idx]}")
plt.show()

# Morlet Limb

In [ ]:
num_clusters = int(np.max(labels) + 1)

fig, ax = plt.subplots(5,5,figsize=(20,30), dpi=400)
for i_cluster in range(num_clusters):
#     idx_cluster = (labels == i_cluster) & (clusterer.probabilities_>0.7)
    idx_cluster = (labels == i_cluster)
    num_fr_in_cluster = len(np.where(idx_cluster)[0])
    
    xtick_idx = np.arange(1, len(freq), 3)
    
    # Create figure
    ax[i_cluster//5, i_cluster%5].imshow(np.mean(limb_power[idx_cluster,:,:], axis=0).T, norm = matplotlib.colors.LogNorm(), 
              vmin=0.01, vmax=np.max(limb_power[:,:,:]), cmap='hot')
    
    # Figure Setting Specific
    if i_cluster%5 == 0: 
        ax[i_cluster//5, i_cluster%5].set(xticks=xtick_idx, xticklabels=np.around(freq[xtick_idx], 1), 
               yticks=np.arange(len(config['limb_labels'])), yticklabels=config['limb_labels'],
               aspect=1.1, xlabel="Frequency", 
               title=f"Avg Power Cluster {i_cluster} ({num_fr_in_cluster} frames)")
    else:
        ax[i_cluster//5, i_cluster%5].set(xticks=xtick_idx, xticklabels=np.around(freq[xtick_idx], 1), yticks=[],
               aspect=1.1, xlabel="Frequency", 
               title=f"Avg Power Cluster {i_cluster} ({num_fr_in_cluster} frames)")
plt.show()